In [2]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
import numpy as np
import cv2
import os
from  sklearn.model_selection import train_test_split
import glob
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
IMAGE_SIZE = 28
IMAGE_CHANNEL = 1

In [4]:
df_train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y_train = df_train['label']
x_train = df_train.drop('label',axis=1).values.reshape(-1,IMAGE_SIZE,IMAGE_SIZE,IMAGE_CHANNEL)
x_test = df_test.values.reshape(-1,IMAGE_SIZE,IMAGE_SIZE,IMAGE_CHANNEL)

In [6]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(IMAGE_SIZE,IMAGE_SIZE,IMAGE_CHANNEL)))
    model.add(Conv2D(32, (3, 3),activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))
    if weights_path and os.path.isfile(weights_path) is True:
        model.load_weights(weights_path)
        
    return model

In [7]:
model = VGG_16(weights_path='Digit_Recognizer_kerasvgg16_weights.h5')
adam = Adam(lr=0.0003)
model.compile(optimizer=adam, loss='categorical_crossentropy')

In [8]:
print(model.input)
print(model.output)
model.summary()

Tensor("zero_padding2d_1_input:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("dense_3/Softmax:0", shape=(?, 10), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 30, 30, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 16, 16, 32)        0         
____________

In [9]:
targets_series = pd.Series(y_train)
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [10]:
print(one_hot_labels)

[[0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


In [11]:
y_train_raw = np.array(one_hot_labels, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

In [12]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(42000, 28, 28, 1)
(42000, 10)
(28000, 28, 28, 1)


In [13]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.03, random_state=1)

In [14]:
model.fit(X_train, Y_train, epochs=5,batch_size=32, validation_data=(X_valid, Y_valid), verbose=1)
model.save_weights('Digit_Recognizer_kerasvgg16_weights.h5')

Train on 40740 samples, validate on 1260 samples
Epoch 1/5
40740/40740 [==============================] - 139s 3ms/step - loss: 0.2398 - val_loss: 0.0804
Epoch 2/5
40740/40740 [==============================] - 139s 3ms/step - loss: 0.0649 - val_loss: 0.0664
Epoch 3/5
40740/40740 [==============================] - 138s 3ms/step - loss: 0.0463 - val_loss: 0.0531
Epoch 4/5
40740/40740 [==============================] - 141s 3ms/step - loss: 0.0329 - val_loss: 0.0489
Epoch 5/5
40740/40740 [==============================] - 139s 3ms/step - loss: 0.0265 - val_loss: 0.0391


In [15]:
model.fit(X_train, Y_train, epochs=5,batch_size=32, validation_data=(X_valid, Y_valid), verbose=1)
model.save_weights('Digit_Recognizer_kerasvgg16_weights.h5')

Train on 40740 samples, validate on 1260 samples
Epoch 1/5
40740/40740 [==============================] - 141s 3ms/step - loss: 0.0236 - val_loss: 0.0386
Epoch 2/5
40740/40740 [==============================] - 144s 4ms/step - loss: 0.0181 - val_loss: 0.0518
Epoch 3/5
40740/40740 [==============================] - 147s 4ms/step - loss: 0.0160 - val_loss: 0.0405
Epoch 4/5
40740/40740 [==============================] - 150s 4ms/step - loss: 0.0117 - val_loss: 0.0571
Epoch 5/5
40740/40740 [==============================] - 146s 4ms/step - loss: 0.0123 - val_loss: 0.0458


In [16]:
y_test = model.predict(x_test,batch_size=32,verbose=1)

28000/28000 [==============================] - 30s 1ms/step


In [29]:
y_pred = np.argmax(y_test,axis=1)
y_pred

array([2, 0, 9, ..., 3, 9, 2])

In [28]:
output_file = "submission_v1.csv"
with open(output_file, 'w') as f :
    f.write('ImageId,Label\n')
    for i in range(len(y_pred)) :
        f.write("".join([str(i+1),',',str(y_pred[i]),'\n']))